# How to create a file output functional pipeline with `toolz.pipe`? 

**Tags:** [python],  [lambda], [toolz], [functional-programming]


A curiously recurring pattern in python, is the need to call the python functions `open`, `write` and `close` to output text to a file. 

How can `toolz.pipe` be used to compose these functions into a resuable functional pipline?


## Imperative implementation

Let's start with the following imperative implementation.

In [2]:
import json
import _io
from dataclasses import dataclass, replace
from toolz import compose_left, do, pipe

DATA = dict(key="json key", value="json value")
TEXT = json.dumps(DATA)
FILENAME1 = "filename1.json"

io = open(FILENAME1, mode='w')                                  # line 1: open
chars = io.write(TEXT)                                          # line 2: write
io.close()                                                      # line 3: close
print(f"Wrote the file `{FILENAME1}` with {chars} characters.") # line 4: print


Wrote the file `filename1.json` with 42 characters.


This code is an imperative implementation, but this is the way that it is commonly done in python. The value returned by each function is at the module scope and vulnerable to being overwritten. 

Instead of having to repeat these 3 function calls for every file that we wish to output, it would be convenient to be able to chain these 3 functons together into a single reusable pipeline for writing ourput files. 

## Object-oriented implementation

This cna be simply and easily implemented simply in object oriented programing, where the object is used to manage the state of the intermediate values returned by each `open` and `write` function call. 


In [8]:

@dataclass
class File:
    """File."""

    name: str
    io: _io.TextIOWrapper = None
    chars: int = 0

    def write(self, text):
        """Write file."""
        self.io = open(self.name, mode='w')
        self.chars = self.io.write(text)
        self.io.close()
        return self

    def show(self):
        """Show file."""
        print(f"Wrote the file `{self.name}` with {self.chars} characters.")


Wrote the file `filename2.json` with 42 characters.



The object oriented implementaiton can be simply ans consicely reused to output any data to a file anywhere in your program as follows;


In [ ]:
FILENAME2 = "filename2.json"

File(FILENAME2).write(TEXT).show()


This set me thinking; what would the functional programming implementatiom look like?
 


## Functional implementation

I have recently read articles about the many benefits of functional programming. 
By programming in a functional style, these 3 funcitons could be composed into a single functional pipeline that could be reused without the need to name the intermediate variables storing the state. Another benefit is that the intermediate states can be made immutable and safely encapsulated using a `dataclass`. 

Using the `toolz.pipe` function, I came up with this implementaiton which produces the same output and side effect (writing file) as the imperative implementation. 


In [9]:
@dataclass(frozen=True)
class File:
    """File."""

    name: str
    io: _io.TextIOWrapper = None
    chars: int = 0

pipe(File("filename3.json"),
    lambda file: replace(file, io=open(file.name, mode='w')),                   # line 1: open
    lambda file: replace(file, chars=file.io.write(TEXT)),                      # line 2: write
    lambda file: do(lambda file: file.io.close(), file),                        # line 3: close
    lambda file: f"Wrote the file `{file.name}` with {file.chars} characters.") # line 4: print


'Wrote the file `filename3.json` with 42 characters.'

This funcitonal implementation of the pipline of fucntion calls, is not as concise or as simple as the imperative or object orented implementations.

I need to understand why this is the case; especially when programming in a functional style promises many benefits.   

This leads to the following questions;

* Is there a better way to implement this using `toolz.pipe`?

    For example;
    - Not using so many lambdas. 
    - Not requiring a `dataclass` to manage the stateful intermediate variables. 
* Are there some other `toolz` library functions that could be used to improve and simplify the implementation (e.g. `curry`, `compose_left`)? 
* Is the `toolz.pipe` function being used correctly and as it was intended to be used?


In [0]:
from toolz import compose_left

compose_left()

In [0]:
from toolz.curried import pipe 

In [0]:
compose_left(lambda x: x**2, lambda x: f"result = {x}")